In [67]:
import datasets

data = datasets.load_dataset("lecslab/glosslm")
data = data['train'].to_pandas()
data

,ID,glottocode,transcription,glosses,translation,metalang_glottocode,is_segmented,source
0,st_arap1274_0,arap1274,heebe,hello.friend,hello friend .,stan1293,yes,sigmorphon_st
1,st_arap1274_0,arap1274,heebe,hello.friend,hello friend .,stan1293,no,sigmorphon_st
2,st_arap1274_1,arap1274,hee,yes,yes/hello .,stan1293,yes,sigmorphon_st
3,st_arap1274_1,arap1274,hee,yes,yes/hello .,stan1293,no,sigmorphon_st
4,st_arap1274_2,arap1274,nooxeihi' nih-'owoh-cee-ceneeni-n,maybe PAST-enjoy-REDUP-sing.at.Sun.Dance-2S,Maybe you enjoyed singing songs at the Sun Dan...,stan1293,yes,sigmorphon_st
...,...,...,...,...,...,...,...,...
451402,langsci369-a0628860cd,,Nu şia ajutat pe nimeni dintre ai săi .,NEG CL.3SG.DAT-has helped LOC=DOM nobody from ...,Intended: `He hasn't helped anybody of his.' *...,stan1293,no,imtvault
451403,langsci369-d018caeb9c,,Şii/*mii(l) ajută pe prieteni .,CL.3SG.REFL.DAT/1SG.DAT-CL.3M.SG.ACC help.3SG ...,Intended: `He is helping his own/my friend.,stan1293,no,imtvault
451404,langsci369-d9f31a29fa,,"Pe prietenii , Ion şiii ajută .","LOC=DOM friends, Ion CL.DAT.3SG.REFL-CL.3M.PL....","His own friends, Ion helps them.",stan1293,no,imtvault
451405,langsci369-0687fce7d6,,Nu şiia trimis pe nimeni*i în ajutori .,NEG CL.3SG.REFL.DAT-has sent LOC=DOM nobody in...,Lit. `He hasn't sent anybody to/as his own aid.,stan1293,no,imtvault


In [68]:
in_dist = ['arap1274', 'uspa1245', 'dido1241']
oo_dist = ['nyan1302', 'natu1246', 'lezg1247']

mask_in_dist = (data['source'] == 'sigmorphon_st') & data['glottocode'].isin(in_dist)
mask_oo_dist = data['glottocode'].isin(oo_dist)
mask_not_in_either = ~(mask_in_dist | mask_oo_dist)

in_dist_data = data[mask_in_dist]
oo_dist_data = data[mask_oo_dist]
train = data[mask_not_in_either]

print(f"Selected {len(in_dist_data)} in-distribution examples, {len(oo_dist_data)} out-of-distribution, leaving {len(train)} remaining")

Selected 128006 in-distribution examples, 8991 out-of-distribution, leaving 314410 remaining


In [69]:
# Create split, stratifying by language and keeping IDs together
from sklearn.model_selection import train_test_split

unique_id_glottocode = in_dist_data[['ID', 'glottocode']].drop_duplicates()
train_ids, test_ids = train_test_split(unique_id_glottocode, test_size=0.15, stratify=unique_id_glottocode['glottocode'])
train_id = data[data['ID'].isin(train_ids['ID'])]
test_id = data[data['ID'].isin(test_ids['ID'])]

test_id.groupby('glottocode')['ID'].nunique()

glottocode
arap1274    7338
dido1241     667
uspa1245    1596
Name: ID, dtype: int64

In [70]:
# Do the same for OOD data
unique_id_glottocode = oo_dist_data[['ID', 'glottocode']].drop_duplicates()
train_ids, test_ids = train_test_split(unique_id_glottocode, test_size=0.15, stratify=unique_id_glottocode['glottocode'])
train_ood = data[data['ID'].isin(train_ids['ID'])]
test_ood = data[data['ID'].isin(test_ids['ID'])]

test_ood.groupby('glottocode')['ID'].nunique()

glottocode
lezg1247    132
natu1246    149
nyan1302    394
Name: ID, dtype: int64

In [71]:
import pandas as pd

train = pd.concat([train, train_id])

In [72]:
split_dataset = datasets.DatasetDict()
split_dataset['eval_id'] = datasets.Dataset.from_pandas(test_id).remove_columns(["__index_level_0__"])
split_dataset['ft_ood'] = datasets.Dataset.from_pandas(train_ood).remove_columns(["__index_level_0__"])
split_dataset['eval_ood'] = datasets.Dataset.from_pandas(test_ood).remove_columns(["__index_level_0__"])
split_dataset['train'] = datasets.Dataset.from_pandas(train).remove_columns(["__index_level_0__"])
split_dataset

DatasetDict({
    eval_id: Dataset({
        features: ['ID', 'glottocode', 'transcription', 'glosses', 'translation', 'metalang_glottocode', 'is_segmented', 'source'],
        num_rows: 19202
    })
    ft_ood: Dataset({
        features: ['ID', 'glottocode', 'transcription', 'glosses', 'translation', 'metalang_glottocode', 'is_segmented', 'source'],
        num_rows: 7642
    })
    eval_ood: Dataset({
        features: ['ID', 'glottocode', 'transcription', 'glosses', 'translation', 'metalang_glottocode', 'is_segmented', 'source'],
        num_rows: 1349
    })
    train: Dataset({
        features: ['ID', 'glottocode', 'transcription', 'glosses', 'translation', 'metalang_glottocode', 'is_segmented', 'source'],
        num_rows: 423214
    })
})

In [73]:
# Sanity check

split_dataset_total_rows = sum(len(dataset) for dataset in split_dataset.values())

if (split_dataset_total_rows != len(data)):
    print("Mismatch!")
else:
    print("Looks good :)")

Looks good :)


In [76]:
split_dataset.push_to_hub("lecslab/glosslm")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/424 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]